In [20]:
import joblib
import numpy as np
import pandas as pd
import json
import requests
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import precision_recall_fscore_support

In [4]:
url = "https://gist.githubusercontent.com/farhaan-settyl/ecf9c1e7ab7374f18e4400b7a3d2a161/raw/f94652f217eeca83e36dab9d08727caf79ebdecf/dataset.json"
response = requests.get(url)
data = json.loads(response.text)

In [10]:
df = pd.DataFrame(data)
df.head()


,externalStatus,internalStatus
0,PORT OUT,Port Out
1,TERMINAL IN,Inbound Terminal
2,PORT IN,Port In
3,Vessel departure from first POL (Vessel name :...,Departure
4,Vessel arrival at final POD (Vessel name : TIA...,Arrival


In [11]:
df['externalStatus'] = df['externalStatus'].str.replace(r'\(.*\)', '')  
df['externalStatus'] = df['externalStatus'].str.lower() 
df.head()

,externalStatus,internalStatus
0,port out,Port Out
1,terminal in,Inbound Terminal
2,port in,Port In
3,vessel departure from first pol (vessel name :...,Departure
4,vessel arrival at final pod (vessel name : tia...,Arrival


In [12]:
label_encoder = LabelEncoder()
df['internalStatus_encoded'] = label_encoder.fit_transform(df['internalStatus'])


In [13]:
X = df['externalStatus']
y = df['internalStatus_encoded']

In [14]:
vectorizer = CountVectorizer()
X_vectorized = vectorizer.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.2, random_state=42)


In [26]:
nn_model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

nn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = nn_model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2, callbacks=[early_stopping])



loss, accuracy = nn_model.evaluate(X_test, y_test)
print(f"Neural Network Accuracy: {accuracy}")

Epoch 1/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.1346 - loss: 2.6613 - val_accuracy: 0.5561 - val_loss: 2.3101
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5353 - loss: 2.1824 - val_accuracy: 0.7551 - val_loss: 1.7082
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6033 - loss: 1.6169 - val_accuracy: 0.7653 - val_loss: 1.1482
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7125 - loss: 1.1934 - val_accuracy: 0.8010 - val_loss: 0.7683
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7787 - loss: 0.9030 - val_accuracy: 0.8367 - val_loss: 0.5743
Epoch 6/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7845 - loss: 0.7723 - val_accuracy: 0.8520 - val_loss: 0.4549
Epoch 7/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8532 - loss: 0.5909 - val_accuracy: 0.8724 - val_loss: 0.3739
Epoch 8/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8750 - loss: 0.4814 - val_accuracy: 0.8980 - val_loss

In [27]:
y_pred = nn_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred_classes, average='weighted')

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1_score}")

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Precision: 0.9752410854451672
Recall: 0.9836734693877551
F1 Score: 0.9789931972789115


C:\Python37\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [31]:
import json
from tensorflow.keras.models import model_from_json


model_json = nn_model.to_json()
with open("nn_model_architecture.json", "w") as json_file:
    json_file.write(model_json)

nn_model.save_weights("nn_model_weights.weights.h5")

print("Neural network model architecture and weights saved successfully.")


with open("nn_model_architecture.json", "r") as json_file:
    loaded_model_json = json_file.read()

loaded_model = model_from_json(loaded_model_json)


loaded_model.load_weights("nn_model_weights.weights.h5")

print("Neural network model loaded successfully.")


Neural network model architecture and weights saved successfully.
Neural network model loaded successfully.
